# Visualize Target Approach

This notebook works for VR target approach experiments where the 2D and 3D stimuli are interleaved (after April 13, 2021)

In [2]:
import os
import sys
import h5py

import pandas as pd
import numpy as np
import panel as pn
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from tqdm.notebook import tqdm
from scipy.stats import sem
from datetime import datetime

sys.path.insert(0, r'..\..')
import paths
import functions_bondjango as bd
import functions_kinematic as fk
import functions_plotting as fp
import functions_preprocessing as fpr
import functions_misc as fm
import functions_data_handling as fd
import functions_vscreens as vs

import importlib
importlib.reload(vs)

line_width = 5

In [3]:
def thresh_function(param, thres):
    return param <= thres

In [4]:
def remove_trials_end_early(data):
    """Parse trial params to only include trials that were run if the experiment ended early"""
    num_completed_trials = data['traces']['trails'].unique().to_numpy()
    print(max(num_completed_trials))

In [ ]:
# define the name to be used for the saved figures
save_name = 'VScreen_interleaved'

In [ ]:
# If this data set was already processed
trial_approaches = pd.read_hdf(r"D:/vscreens/" + save_name, 'data')

In [6]:
search_params = {'result': 'test',
                 'lighting': 'normal',
                 'rig': 'VScreen',
                 'analysis_type': 'preprocessing',
                 'ltdate': '2021-03-22T00-00-00',
#                  'notes': dimensionality
                }

search_string = ', '.join("{}:{}".format(key,val) for (key,val) in search_params.items())
data = vs.load_VScreens_datasets(search_string, exclusion="removedtracker")

J:\Drago Guggiana Nilo\Prey_capture\AnalyzedData\11_23_2020_11_16_30_VScreen_DG_200526_d_test_startTS_CW_preproc.hdf5
2020-11-23
J:\Drago Guggiana Nilo\Prey_capture\AnalyzedData\11_17_2020_14_19_03_VScreen_DG_200526_d_test_3D_preproc.hdf5
2020-11-17
J:\Drago Guggiana Nilo\Prey_capture\AnalyzedData\11_17_2020_12_51_11_VScreen_DG_200526_c_test_3D_preproc.hdf5
2020-11-17
J:\Drago Guggiana Nilo\Prey_capture\AnalyzedData\11_17_2020_11_19_26_VScreen_DG_200526_b_test_3D_preproc.hdf5
2020-11-17
J:\Drago Guggiana Nilo\Prey_capture\AnalyzedData\11_17_2020_09_49_42_VScreen_DG_200526_a_test_3D_preproc.hdf5
2020-11-17
J:\Drago Guggiana Nilo\Prey_capture\AnalyzedData\11_16_2020_15_09_05_VScreen_DG_200526_d_test_3D_preproc.hdf5
2020-11-16
J:\Drago Guggiana Nilo\Prey_capture\AnalyzedData\11_16_2020_13_34_54_VScreen_DG_200526_c_test_3D_preproc.hdf5
2020-11-16
J:\Drago Guggiana Nilo\Prey_capture\AnalyzedData\11_16_2020_12_03_30_VScreen_DG_200526_b_test_3D_preproc.hdf5
2020-11-16
J:\Drago Guggiana Nilo\P

In [ ]:
exp_manipulation_params = ['color_target', 'scale', 'speed']
df_columns = ['date', 'animal', 'bin', 'trial_num', 'target_color', 'speed', 'scale', 'approaches', 'has_approach']

# encounter_criteria
approach_criteria = {'2D': {'distance_threshold':0.05, 'speed_minimum': 0.10, 'start_distance': 0.05, 'window': 2},
                     '3D': {'distance_threshold':0.10, 'speed_minimum': 0.10, 'start_distance': 0.05, 'window': 2},
                     'cricket': {'distance_threshold':0.10, 'speed_minimum': 0.10, 'start_distance': 0.05, 'window': 2}
                     }

# This lookup list matches the target types as defined in the Unity scene
target_lookup = ['2D', '2D', '2D', '3D', '3D', '3D', 'cricket']

In [ ]:
# For later visualization
all_valid_approaches = []
all_approaches = []
bin_duration = 10    # minutes

test_ds = data[3:4]

# Make a new data frame
approach_df = pd.DataFrame(columns=df_columns)

for exp_num, exp in tqdm(enumerate(test_ds), total=len(test_ds)):

    # Calculate time bins for each data point in the experiment
    exp['traces']['bin'] = vs.calculate_bins(exp['traces']['time_vector'], bin_duration)

    # Calculate the distance travelled between each time point in the experiment
    exp['traces']['distance_travelled'] = fk.accumulated_distance(exp['traces'][['mouse_x_m', 'mouse_y_m']].to_numpy())
        
#     # Extract the approaches in this experiment
#     experiment_approaches = pd.DataFrame(columns=df_columns)
#     experiment_approaches, exp_valid_approaches, all_exp_approaches = vs.extract_experiment_approaches(exp, 
#                                                                                                   exp_manipulation_params, 
#                                                                                                   crit, 
#                                                                                                   thresh_function,
#                                                                                                   experiment_approaches
#                                                                                                  )
    
#     approach_df = approach_df.append(experiment_approaches, ignore_index=True)
#     all_valid_approaches.append(exp_valid_approaches)
#     all_approaches.append(all_exp_approaches)
                             
        
# approach_df.to_hdf(r"D:/vscreens/" + save_name, 'data')